In [1]:
import os
import pickle
from glob import glob

In [2]:
import numpy as np

import torch

%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
import sys
sys.path.append("../src")

In [4]:
from utils.plots import input2image
from utils import plots

from my_model import get_ilsvrc2012
from utils.load_model import get_model

from utils.tensortracker import TensorTracker

from utils.performance_model import measure_performance

from utils.config import dict_from_config_file

In [5]:
n_classes = 1000

In [6]:
model_names = [
    ("resnet34-skip4", "resnet"),
    ("resnet34-skip", "resnet34-1"),
    ("resnet34-skip2", "resnet34-2"),
    ("resnet34-skip3", "resnet34-3"),
    ("resnet34-plain4", "plainnet"),
    ("resnet34-plain", "plainnet34-1"),
    ("resnet34-plain2", "plainnet34-2"),
    ("resnet34-plain3", "plainnet34-3"),
]

In [7]:
analysis_root = "/data2/genta/resnet/analysis/"
# rf_root = "/mnt/nas3/lab_member_directories/2021_genta/resnet/e_receptive_field/"
rf_root = "/mnt/nas5/lab_member_directories/2021_genta/resnet/e_receptive_field/"

In [8]:
in_dir = "20201216/"
numberinclass = {}
in_path = os.path.join(in_dir, "data_resnets.pkl")
with open(in_path, "rb") as f:
    numberinclass["resnet"] = pickle.load(f)
in_path = os.path.join(in_dir, "data_plainnets.pkl")
with open(in_path, "rb") as f:
    numberinclass["plainnet"] = pickle.load(f)

In [9]:
numberinclass["resnet"]["model_names"]

['20200409_resnet34', 'resnet34-skip_', 'resnet34-skip2_', 'resnet34-skip3_']

In [10]:
numberinclass["plainnet"]["model_keys"]

['resnet34-plain4', 'resnet34-plain', 'resnet34-plain2', 'resnet34-plain3']

In [11]:
class ChannelCutter(object):
    def __init__(self, model):
        self.model = model
        self.original_weights = {}
    
    def add_weights(self, name, weight):
        with torch.no_grad():
            if name not in self.original_weights:
                self.original_weights[name] = weight.clone()
                
                
    def set_original_all_weights(self):
        with torch.no_grad():
            for name in self.original_weights:
                layer_name = name.split(".")[0]
                self.set_original_weights(layer_name)
                
    def set_original_weights(self, layer_name):
        with torch.no_grad():
            if layer_name == "fc":
                key = "{}.weight".format(layer_name)
                model.fc.weight.data = self.original_weights[key].data.clone()
                key = "{}.bias".format(layer_name)
                model.fc.bias.data = self.original_weights[key].data.clone()
            else:
                raise ValueError(layer_name)

    def cut_channel_output(self, layer_name, ch):
        with torch.no_grad():
            if layer_name == "fc":
                weight = self.model.fc.weight
                zeros = torch.zeros_like(weight[:, ch])
                self.add_weights("fc.weight", weight)
                self.model.fc.weight[:, ch] = zeros
                bias = self.model.fc.bias
                zeros = torch.zeros_like(bias[ch])
                self.add_weights("fc.bias", bias)
                self.model.fc.bias[ch] = zeros
            else:
                raise ValueError(layer_name)

In [12]:
from utils.performance_model import measure_performance

In [14]:
dataset = get_ilsvrc2012(mode="test")
batch_size = 512
gpu = 1

channelcut_performances = {}

In [22]:
%%time

m_key = "resnet"

n_sample = 5
model_i = 0
block_id = 2
key_layer = "layer4"
target_numberinclass = numberinclass[m_key]
model = get_model(target_numberinclass["model_keys"][model_i])
ch_cutter = ChannelCutter(model)

key = "{}-{}-{}".format(target_numberinclass["model_names"][model_i], key_layer, block_id)
target_unique_lens = target_numberinclass["data"][key][2]

sorted_channels = np.argsort(target_unique_lens)[::-1]

channelcut_performances[m_key] = {}

ch_cutter.set_original_all_weights()
channelcut_performances[m_key][-1] = measure_performance(ch_cutter.model, dataset, batch_size=batch_size, gpu=gpu, with_acts=True)

for ch in sorted_channels[:n_sample]:
    ch_cutter.set_original_all_weights()
    ch_cutter.cut_channel_output("fc", ch)
    channelcut_performances[m_key][ch] = measure_performance(ch_cutter.model, dataset, batch_size=batch_size, gpu=gpu, with_acts=True)

for ch in sorted_channels[-n_sample:]:
    ch_cutter.set_original_all_weights()
    ch_cutter.cut_channel_output("fc", ch)
    channelcut_performances[m_key][ch] = measure_performance(ch_cutter.model, dataset, batch_size=batch_size, gpu=gpu, with_acts=True)

/home/genta/.pyenv/versions/dlb2-pytorch/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


CPU times: user 9h 2min 58s, sys: 3min 17s, total: 9h 6min 16s
Wall time: 1h 3min 24s


In [23]:
target_numberinclass = numberinclass[m_key]
key = "{}-{}-{}".format(target_numberinclass["model_names"][model_i], key_layer, block_id)
target_unique_lens = target_numberinclass["data"][key][2]
for ch, data in channelcut_performances[m_key].items():
    if ch < 0:
        continue
    tmp_data0 = channelcut_performances[m_key][-1][0]
    tmp_data = data[0]
    print("ch{:003}, count:{:03}, loss {:.5E}, top1 acc {:.3f}".format(
        ch, target_unique_lens[ch], tmp_data[0].mean() - tmp_data0[0].mean(), tmp_data[1].mean() - tmp_data0[1].mean()))

ch206, count:051, loss 5.00900E-04, top1 acc -0.032
ch428, count:045, loss 1.13209E-03, top1 acc -0.065
ch212, count:045, loss 7.01139E-04, top1 acc -0.038
ch467, count:043, loss 1.66859E-03, top1 acc -0.050
ch435, count:042, loss 1.75840E-03, top1 acc -0.067
ch081, count:009, loss 1.64681E-03, top1 acc -0.040
ch430, count:008, loss 2.45062E-03, top1 acc -0.091
ch048, count:008, loss 2.61768E-03, top1 acc -0.012
ch506, count:007, loss 2.91511E-03, top1 acc -0.068
ch415, count:007, loss 4.63297E-03, top1 acc -0.140


In [24]:
%%time

m_key = "plainnet"

n_sample = 5
model_i = 0
block_id = 2
key_layer = "layer4"
target_numberinclass = numberinclass[m_key]
model = get_model(target_numberinclass["model_keys"][model_i])
ch_cutter = ChannelCutter(model)

key = "{}-{}-{}".format(target_numberinclass["model_names"][model_i], key_layer, block_id)
target_unique_lens = target_numberinclass["data"][key][2]

sorted_channels = np.argsort(target_unique_lens)[::-1]

channelcut_performances[m_key] = {}

ch_cutter.set_original_all_weights()
channelcut_performances[m_key][-1] = measure_performance(ch_cutter.model, dataset, batch_size=batch_size, gpu=gpu, with_acts=True)

for ch in sorted_channels[:n_sample]:
    ch_cutter.set_original_all_weights()
    ch_cutter.cut_channel_output("fc", ch)
    channelcut_performances[m_key][ch] = measure_performance(ch_cutter.model, dataset, batch_size=batch_size, gpu=gpu, with_acts=True)

for ch in sorted_channels[-n_sample:]:
    ch_cutter.set_original_all_weights()
    ch_cutter.cut_channel_output("fc", ch)
    channelcut_performances[m_key][ch] = measure_performance(ch_cutter.model, dataset, batch_size=batch_size, gpu=gpu, with_acts=True)

CPU times: user 9h 3min 36s, sys: 3min 12s, total: 9h 6min 49s
Wall time: 1h 2min 57s


In [25]:
target_numberinclass = numberinclass[m_key]
key = "{}-{}-{}".format(target_numberinclass["model_names"][model_i], key_layer, block_id)
target_unique_lens = target_numberinclass["data"][key][2]
for ch, data in channelcut_performances[m_key].items():
    if ch < 0:
        continue
    tmp_data0 = channelcut_performances[m_key][-1][0]
    tmp_data = data[0]
    print("ch{:003}, count:{:03}, loss {:.5E}, top1 acc {:.3f}".format(
        ch, target_unique_lens[ch], tmp_data[0].mean() - tmp_data0[0].mean(), tmp_data[1].mean() - tmp_data0[1].mean()))

ch466, count:040, loss 2.57773E-03, top1 acc -0.098
ch037, count:038, loss 2.31040E-03, top1 acc 0.002
ch089, count:038, loss 5.20002E-04, top1 acc 0.002
ch104, count:038, loss 1.74673E-03, top1 acc -0.035
ch441, count:037, loss 9.78314E-04, top1 acc -0.046
ch496, count:007, loss 3.05658E-03, top1 acc -0.028
ch218, count:007, loss 2.48525E-03, top1 acc -0.039
ch375, count:007, loss 2.04332E-03, top1 acc -0.073
ch197, count:007, loss 3.98087E-03, top1 acc 0.017
ch209, count:006, loss 2.12089E-03, top1 acc -0.029


In [42]:
m_key = "resnet"
target_numberinclass = numberinclass[m_key]
key = "{}-{}-{}".format(target_numberinclass["model_names"][model_i], key_layer, block_id)
target_unique_lens = target_numberinclass["data"][key][2]
for ch, data in channelcut_performances[m_key].items():
    if ch < 0:
        continue
    tmp_data0 = channelcut_performances[m_key][-1][0]
    tmp_data = data[0]
    print("ch{:003}, count:{:03}, loss {:.5E}, top1 acc {:.3f}".format(
        ch, target_unique_lens[ch], tmp_data[0].mean() - tmp_data0[0].mean(), tmp_data[1].mean() - tmp_data0[1].mean()))

ch206, count:051, loss 5.00900E-04, top1 acc -0.032
ch428, count:045, loss 1.13209E-03, top1 acc -0.065
ch212, count:045, loss 7.01139E-04, top1 acc -0.038
ch467, count:043, loss 1.66859E-03, top1 acc -0.050
ch435, count:042, loss 1.75840E-03, top1 acc -0.067
ch081, count:009, loss 1.64681E-03, top1 acc -0.040
ch430, count:008, loss 2.45062E-03, top1 acc -0.091
ch048, count:008, loss 2.61768E-03, top1 acc -0.012
ch506, count:007, loss 2.91511E-03, top1 acc -0.068
ch415, count:007, loss 4.63297E-03, top1 acc -0.140


In [28]:
channelcut_performances_cum = {}

In [46]:
%%time

m_key = "resnet"

n_sample = 5
model_i = 0
block_id = 2
key_layer = "layer4"
target_numberinclass = numberinclass[m_key]
model = get_model(target_numberinclass["model_keys"][model_i])
ch_cutter = ChannelCutter(model)

key = "{}-{}-{}".format(target_numberinclass["model_names"][model_i], key_layer, block_id)
target_unique_lens = target_numberinclass["data"][key][2]

sorted_channels = np.argsort(target_unique_lens)[::-1]

channelcut_performances_cum[m_key] = {}

for cnt, ch in enumerate(sorted_channels[:n_sample]):
    ch_cutter.cut_channel_output("fc", ch)
    if cnt == 0:
        continue
    channelcut_performances_cum[m_key][ch] = measure_performance(ch_cutter.model, dataset, batch_size=batch_size, gpu=gpu, with_acts=True)

/home/genta/.pyenv/versions/dlb2-pytorch/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


CPU times: user 3h 26min 19s, sys: 1min 12s, total: 3h 27min 32s
Wall time: 24min 42s


In [47]:
%%time

m_key = "plainnet"

n_sample = 5
model_i = 0
block_id = 2
key_layer = "layer4"
target_numberinclass = numberinclass[m_key]
model = get_model(target_numberinclass["model_keys"][model_i])
ch_cutter = ChannelCutter(model)

key = "{}-{}-{}".format(target_numberinclass["model_names"][model_i], key_layer, block_id)
target_unique_lens = target_numberinclass["data"][key][2]

sorted_channels = np.argsort(target_unique_lens)[::-1]

channelcut_performances_cum[m_key] = {}

for cnt, ch in enumerate(sorted_channels[:n_sample]):
    ch_cutter.cut_channel_output("fc", ch)
    if cnt == 0:
        continue
    channelcut_performances_cum[m_key][ch] = measure_performance(ch_cutter.model, dataset, batch_size=batch_size, gpu=gpu, with_acts=True)

CPU times: user 3h 21min 3s, sys: 1min 5s, total: 3h 22min 9s
Wall time: 23min 25s


In [48]:
list(channelcut_performances[m_key])

[-1, 466, 37, 89, 104, 441, 496, 218, 375, 197, 209]

In [49]:
m_key = "resnet"
for cnt, ch in enumerate(channelcut_performances[m_key]):
    if cnt > 1:
        break
    channelcut_performances_cum[m_key][ch] = channelcut_performances[m_key][ch]
    
m_key = "plainnet"
for cnt, ch in enumerate(channelcut_performances[m_key]):
    if cnt > 1:
        break
    channelcut_performances_cum[m_key][ch] = channelcut_performances[m_key][ch]

In [50]:
list(channelcut_performances_cum[m_key])

[37, 89, 104, 441, -1, 466]

In [51]:
m_key = "resnet"
target_numberinclass = numberinclass[m_key]
key = "{}-{}-{}".format(target_numberinclass["model_names"][model_i], key_layer, block_id)
target_unique_lens = target_numberinclass["data"][key][2]
sorted_channels = np.argsort(target_unique_lens)[::-1]
for cnt, ch in enumerate(sorted_channels):
    if cnt >= len(channelcut_performances_cum[m_key]) - 1:
        break
    data = channelcut_performances_cum[m_key][ch]
    tmp_data0 = channelcut_performances_cum[m_key][-1][0]
    tmp_data = data[0]
    print("ch{:003}, count:{:03}, loss {:.5E}, top1 acc {:.3f}".format(
        ch, target_unique_lens[ch], tmp_data[0].mean() - tmp_data0[0].mean(), tmp_data[1].mean() - tmp_data0[1].mean()))

ch206, count:051, loss 5.00900E-04, top1 acc -0.032
ch428, count:045, loss 1.58781E-03, top1 acc -0.123
ch212, count:045, loss 2.15097E-03, top1 acc -0.147
ch467, count:043, loss 3.76409E-03, top1 acc -0.157
ch435, count:042, loss 5.56130E-03, top1 acc -0.192


In [52]:
m_key = "plainnet"
target_numberinclass = numberinclass[m_key]
key = "{}-{}-{}".format(target_numberinclass["model_names"][model_i], key_layer, block_id)
target_unique_lens = target_numberinclass["data"][key][2]
sorted_channels = np.argsort(target_unique_lens)[::-1]
for cnt, ch in enumerate(sorted_channels):
    if cnt >= len(channelcut_performances_cum[m_key]) - 1:
        break
    data = channelcut_performances_cum[m_key][ch]
    tmp_data0 = channelcut_performances_cum[m_key][-1][0]
    tmp_data = data[0]
    print("ch{:003}, count:{:03}, loss {:.5E}, top1 acc {:.3f}".format(
        ch, target_unique_lens[ch], tmp_data[0].mean() - tmp_data0[0].mean(), tmp_data[1].mean() - tmp_data0[1].mean()))

ch466, count:040, loss 2.57773E-03, top1 acc -0.098
ch037, count:038, loss 5.06831E-03, top1 acc -0.151
ch089, count:038, loss 5.58058E-03, top1 acc -0.153
ch104, count:038, loss 6.93865E-03, top1 acc -0.192
ch441, count:037, loss 7.68064E-03, top1 acc -0.216


In [53]:
out_dir = "20201217"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [54]:
out_path = os.path.join(out_dir, "channelcut_performances.pkl")
with open(out_path, "wb") as f:
    pickle.dump(channelcut_performances, f)
    
out_path = os.path.join(out_dir, "channelcut_performances_cum.pkl")
with open(out_path, "wb") as f:
    pickle.dump(channelcut_performances_cum, f)

In [57]:
channelcut_performances["resnet"][-1][0][0].mean()

1.1581143749003509

In [58]:
channelcut_performances["resnet"][206][0][0].mean()

1.158615274696934

In [59]:
channelcut_performances["plainnet"][-1][0][0].mean()

1.327886345435162

In [59]:
channelcut_performances["plainnet"][-1][0][0].mean()

1.327886345435162

In [61]:
# top 1 acc
channelcut_performances["resnet"][-1][0][1].mean()

71.4292357308524

In [62]:
# top 1 acc
channelcut_performances["plainnet"][-1][0][1].mean()

67.67027679754763

In [63]:
# top 1 acc
channelcut_performances["resnet"][-1][0][1].mean()

71.4292357308524